<a href="https://colab.research.google.com/github/MahdiJahanbakht/cat-dog-classification-on-colab/blob/master/3_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learn to beat cat-dog discrimination problem :)
In this notebook I provided you with the code that does the transfer learning on a pre-trained network to train a model that solves the cat-dog classification problem. We will use a resnet network trained on  [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html).  
>**Hint:** imageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep convolutional neural networks.



 >**Note:**
If you want to benefit from the googl's amazing free **Tesla K80** gpu, first check that you are actually using **gpu supporting ** **Colab** note book.  
To do so click on `Runtime>Change runtime type` then select **Python3** from `[Runtime Type]` and **GPU** from `[Hardware accelerator]`.

First, we have to mount  our **google drive**.  
If it is your first time to run this code, you have to do as the following:
* First run the block
* By doing so, it will appear a link in the output. Click on it
* Then, choose your account and in the next window allow **Google Drive File Stream** to do what ever it wants :)
* In the next page copy the verification code provided for you in the box that has been appeared below the aforementioned link in the output and press **Enter**
This would do the trick and mount your drive at `/content/drive`.

### Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Check if data is accessible

In [0]:
dataDir = '/content/drive/My Drive/Deep_Learning/Datasets/Cat_Dog_data'
!ls '$dataDir'

# Installing Pytorch
The code for installing the latest stable release of **pytorch** and **torchvision** on **Linux** system with **python >= 3.6** and **cuda 9.0** using **pip** is as follows([Link](https://pytorch.org/))
In colab the code that pytorch site suggets is slightly different. It is using `pip`, and`-q` is required to install packages in Colab Vm

In [0]:
!pip install -q torch torchvision

### Check **GPU** settings

In [0]:
import torch
# Check wether cuda is available or not
print('cuda availability:',torch.cuda.is_available())
# Get device name
print('device name:',torch.cuda.get_device_name(0))
# Gets the the major and minor cuda capability of the device
print('cuda capability:',torch.cuda.get_device_capability(torch.cuda.current_device()))
# Check the current GPU memory usage by 
# tensors in bytes for a given device(should be zero if nothing is running)
print('allocated memory:',torch.cuda.memory_allocated())
# Check the current GPU memory managed by the
# caching allocator in bytes for a given device(should be zero if nothing is running)
print('cached memory:',torch.cuda.memory_cached())

In order to release cache and freeup some memory you can use this code

In [0]:
# Releases all unoccupied cached memory currently held by
# the caching allocator so that those can be used in other
# GPU application and visible in nvidia-smi
torch.cuda.empty_cache()

### Import Required Libraries

In [0]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import time

import matplotlib.pyplot as plt
# import helper

from collections import OrderedDict
import time

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Make transforms

Most of the torchvision's pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are [0.485, 0.456, 0.406] and the standard deviations are [0.229, 0.224, 0.225]. [Link](https://pytorch.org/docs/0.3.0/torchvision/models.html#id5)

In [0]:
baseDir = '/content/drive/My Drive/Deep_Learning/'
dataDir = baseDir + 'Datasets/Cat_Dog_data/'
runDir = baseDir + 'Cat_Dog_run/'

# we used batch size of 64 images for batch learning
batch_size = 64

# For traing data we used Data Augmentation of the form of transforms that
# randomly rotate, scale and crop, then flip images
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

# For test data we did'nt use any form of Data Augmentation and just used
# resize and center crop to adjust our images for input
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
# Define train and test folders located on google drive
trainset = datasets.ImageFolder(dataDir+'train', transform=train_transforms)
testset = datasets.ImageFolder(dataDir+'test',transform=test_transforms)

# Define batch loaders
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size)

## Model Construction
In this block we define our model. To do thar we first download a pre-trained model from the models available through **torchvision**. Then, freeze all the trained weights and just replace the classification layer, and train just that layer's weights.  
Here we used **ResNet50** that is not a very big network, compared to others. if you print the model,  there is a **fc** layer at the end that works as classifier. We just replace that layer with our personalized classifier.  
Something to remember is that fc layers input is 2048 in size, and it's output is of the size 1000 that is a require ment of **Imagenet** (that has 1000 classe).  
`(fc): Linear(in_features=2048, out_features=1000, bias=True)`  
We changed that to only 2, because we just have two classes (cat and dog)

>**Caution:** If you plan to use **CPU/GPU**, you should move your model and all the inputls and labels to cpu or gpu accordingly. the default is cpu. To do that you can use `.to()` method of torch tensors.

In [0]:
# Use Gpu if it's available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Download pre-trained model
model = models.resnet50(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

# Design our desired classification layer
# LogSofmax goes with NLLLoss
classifier = nn.Sequential(nn.Linear(2048,512),
                          nn.ReLU(),
                          nn.Dropout(p=0.2),
                          nn.Linear(512,2),
                          nn.LogSoftmax(dim=1))
model.fc = classifier

# Define cost function aas the negative log likelihood loss. It is useful to train a classification problem with C classes
criterion = nn.NLLLoss()
# Adam optimizer with learning rate of 0.003
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
# Mode our model to the selected device above
model.to(device)

## Learn To correctly classify adorable creatures :D

In [0]:
# Define our epochs
epochs = 2
# Is used in resume Learning(Next Notebook)
start_epoch = 0
# To count how many steps we gone so far
steps = 0
# Trach training loss
training_loss = 0
# Save checkpoint every n steps to keep track of our models
save_freq = 5

train_losses, test_losses = [], []

for epoch in range(epochs):
    # Load next batch
    for inputs, labels in trainloader: 
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Very important. to clear gradients of previous step   
        optimizer.zero_grad()
        # Do the forward pass
        logps = model.forward(inputs)
        # Calculate cost function
        loss = criterion(logps, labels)
        # Compute gradients
        loss.backward()
        # Update weights
        optimizer.step()
        
        # Sum up all the costs
        training_loss += loss.item()
        
        ##################### Evaluation #######################
        if steps % save_freq == 0:
            eval_loss = 0
            accuracy = 0
            # Do not update weights in evaluation mode and bypass dropout
            model.eval()
            # Do not compute gradients. Speeds up the algorithm alot
            with torch.no_grad():
                # Load test data
                for inputs, labels in testloader:
                    # Move test tensors to device
                    inputs, labels = inputs.to(device), labels.to(device)
                    # Do the forward pass
                    logps = model.forward(inputs)
                    # Compute cost
                    batch_loss = criterion(logps, labels)
                    # Sum up all the costs
                    eval_loss += batch_loss.item()
                    
                    # Calculate accuracy. As you may remember we done LogSoftmax
                    # in the last layer. So, to actual probabolities we should
                    # calculate exp of output values
                    ps = torch.exp(logps)
                    # Select the best class
                    top_p, top_class = ps.topk(1, dim=1)
                    # Reshape our labels to match top_class shape
                    equals = top_class == labels.view(*top_class.shape)
                    # Change the type of equals to float tensor
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            # Keep track of train and test losses for each batch
            mean_train_loss = training_loss/save_freq
            mean_eval_loss = eval_loss/len(testloader)
            mean_acc = accuracy/len(testloader)
            
            train_losses.append(mean_train_loss)
            test_losses.append(mean_eval_loss)
            
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {mean_train_loss:.3f}.. "
                  f"Test loss: {mean_eval_loss:.3f}.. "
                  f"Test accuracy: {mean_acc:.3f}")
            
            # Save chechpoint
            # A checkpoint is a python dictionary that typically includes the following
            # The network structure: input and output sizes and Hidden layers to be able to reconstruct the model at loading time
            # The model state dict : includes parameters of the network layers that is learned during training, 
            # The optimizer state dict : In case you are saving the latest checkpoint to continue training later, you need to save the optimizer’s state as well
            # Additional info: You may need to store additional info, like number of epochs and your class to index mapping in your checkpoint
            
            checkpoint = {'model': classifier,
                          'epoch': epoch + start_epoch,
                          'state_dict': model.state_dict(),
                          'optimizer' : optimizer.state_dict(),
                          'train_loss': mean_train_loss,
                          'test_loss': mean_eval_loss,
                          'accuracy' : mean_acc}

            torch.save(checkpoint, runDir + 'acc-{:.3f}_loss-{:.3f}_epoch-{}_checkpoint.pth.tar'.format(mean_acc, mean_eval_loss, epoch))

            # Clear train losses
            training_loss = 0
            # change bach to learning mode
            model.train()

In [0]:
plt.plot(train_losses, label = 'Train loss')
plt.plot(test_losses, label = 'Test loss')
plt.legend(frameon = False)